In [1]:
import cv2 as cv
import numpy as np
import tensorflow
import keras
from keras import layers as L
import ssl
import os

In [2]:
cv.

<module 'cv2.version' from '/Users/arya/Documents/CodeProjects/RCJ/env/lib/python3.12/site-packages/cv2/version.py'>

In [2]:
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
def build_model(input_shape, num_classes=3):
    """ Inputs """
    inputs = L.Input(input_shape)

    """ Backbone """
    backbone = keras.applications.MobileNetV2(
        include_top=False,
        weights="imagenet",
        input_tensor=inputs,
        alpha=1.0
    )
    backbone.trainable = True
    # backbone.summary()

    """ Detection Head """
    x = backbone.output
    x = L.Conv2D(256, kernel_size=1, padding="same")(x)
    x = L.BatchNormalization()(x)
    x = L.Activation("relu")(x)
    x = L.GlobalAveragePooling2D()(x)
    x = L.Dropout(0.5)(x)
    bbox = L.Dense(4, activation="sigmoid", name="bbox")(x)
    label = L.Dense(num_classes, activation="softmax", name="label")(x)

    """ Model """
    model = keras.models.Model(inputs=[inputs], outputs=[bbox, label])
    return model

In [4]:
def load_model():
    model_path = "./helperScripts/model/model.weights.h5"
    
    model = build_model((218, 384, 3))
    model.load_weights(model_path)

    return model

In [5]:
def build_blob_detector():
    params = cv.SimpleBlobDetector_Params()

    params.filterByArea = True
    params.minArea = 50
    params.maxArea = 100000
    
    params.filterByCircularity = True
    params.minCircularity = 0.45
    
    params.filterByInertia = True
    params.minInertiaRatio = 0.4

    return cv.SimpleBlobDetector_create(params)

In [6]:
def get_prediction(binary_result):
    cv.imwrite("img.jpg", cv.resize(binary_result, (384, 218)))
    img = cv.imread("img.jpg")
    os.remove("img.jpg")

    x = cv.resize(img, (384, 218))
    x = (x - 127.5) / 127.5
    x = np.expand_dims(x, axis=0)

    pred_bbox, label = model.predict(x, verbose=0)
    
    label_found = ""
    color = (221, 221, 221)
    if label[0][0] >= label[0][1] and label[0][0] >= label[0][2]:
        label_found = "H"
        color = (79, 42, 244)
    elif label[0][1] >= label[0][0] and label[0][1] >= label[0][2]:
        label_found = "S"
        color = (21, 64, 47)
    elif label[0][2] >= label[0][0] and label[0][2] >= label[0][1]:
        label_found = "U"
        color = (197, 73, 245)

    max_confidence = max(label[0][0], label[0][1], label[0][2])
    if not max_confidence >= .85:
        label_found = "/"
        color = (0, 0, 0)

    return label_found, max_confidence, color

In [7]:
def get_colors(frame, detector):
    keypoints = detector.detect(frame)

    detected = []
    if len(keypoints) != 0:
        for keypoint in keypoints:
            color = frame[int(keypoint.pt[1]), int(keypoint.pt[0])]
            blue = color[0]
            green = color[1]
            red = color[2]

            if (red / green) + (red / blue) >= 3.5:
                detected.append("Red")
            if (green / blue) + (green / red) >= 2.7:
                detected.append("Green")
            if (green * red) / blue >= 400:
                detected.append("Yellow")

    return keypoints, detected

In [8]:
vid = cv.VideoCapture(0)
model = load_model()
detector = build_blob_detector()

while True:
    ret, frame = vid.read()

    grey = cv.cvtColor(frame, cv.COLOR_RGB2GRAY)
    ret, binary_result = cv.threshold(grey, 127, 255, cv.THRESH_BINARY)

    
    label_found, max_confidence, color = get_prediction(binary_result)
    keypoints, detected = get_colors(frame, detector)
    
    font_size = 0.5
    letter_text = f"{label_found} - {max_confidence:.3f}"
    color_text = " & ".join(detected) if len(detected) > 0 else "None"
    
    cv.putText(frame, letter_text, (5, 20), cv.FONT_HERSHEY_SIMPLEX, font_size, color, 2)
    cv.putText(frame, color_text, (5, 60), cv.FONT_HERSHEY_SIMPLEX, font_size, (255, 0, 0), 2)

    cv.imshow("Normal", frame)

    if cv.waitKey(1) & 0xFF == ord('q'):
        break

vid.release()
cv.destroyAllWindows()

2024-05-06 07:56:41.257 python[35780:4513551] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
/var/folders/4w/7mmrnj315pzgcqvpx7n96sf40000gn/T/ipykernel_35780/2947174705.py:6: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  backbone = keras.applications.MobileNetV2(
/var/folders/4w/7mmrnj315pzgcqvpx7n96sf40000gn/T/ipykernel_35780/3158661915.py:16: RuntimeWarning: overflow encountered in scalar multiply
  if (green * red) / blue >= 400:
